# Fine-tune de Mistral 7B para Chatbot de suporte em Ecommerce

### Instalação de requirimentos

In [1]:
#pip install transformers==4.34.0
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 54.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.1
    Uninstalling transformers-4.38.1:
      Successfully uninstalled transformers-4.38.1


In [2]:
!pip install -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 7.3 MB/s eta 0:00:00


In [3]:
!pip install bitsandbytes -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 14.3 MB/s eta 0:00:00


In [4]:
!pip install peft -U
!pip install -U datasets
!pip install -qU trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.4 MB/s eta 0:00:00


### Carregar o Modelo

In [5]:
import torch
# Verificar se a GPU é compativel com bfloat16
print(torch.cuda.get_device_capability())

(8, 0)


In [6]:
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer

model_id = "mistralai/Mistral-7B-Instruct-v0.1"

bits_and_bytes_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    #bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

mistral_7B = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bits_and_bytes_config,
    device_map="auto"
)

mistral_tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

### Pós processamento do modelo
Para permitir o treino do modelo temos que congelar o modelo e colocar a layer-norm em float32, para dar estabilidade ao modelo. O mesmo se aplica para o output da última layer.

In [7]:
import torch.nn as nn
#vamos só treinar o adpater
for param in mistral_7B.parameters():
  param.requires_grad = False  # congelar o modelo
  if param.ndim == 1:
    #colocar a layor-norm que tem poucos parâmetros em float32
    param.data = param.data.to(torch.float32)

mistral_7B.gradient_checkpointing_enable()
mistral_7B.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
mistral_7B.lm_head = CastOutputToFloat(mistral_7B.lm_head)
mistral_tokenizer.add_special_tokens({'pad_token': '[PAD]'}) #garantir consistência nos input sizes

1

In [ ]:
import transformers
from torch.cuda.amp import autocast


text =  "<s>[INST] Tu és um agente de suporte ao cliente de um site de ecommerce. Fala em Português de Portugal. Não repitas que és um agente de suporte ao cliente. Apresenta-te?[/INST]"
inputs = mistral_tokenizer(text, return_tensors="pt")

inputs = inputs.to('cuda') #Colocar os inputs na GPU
with autocast(): #Permitir computação de precisão mista
    outputs = mistral_7B.generate(**inputs, max_new_tokens=150)

print(mistral_tokenizer.decode(outputs[0], skip_special_tokens=True))

### Aplicar LoRA
Utilizar low-rank adapters (LoRA) quantizado (QLoRA)

In [9]:
def print_trainable_parameters(model):

    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [10]:
print(mistral_7B)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
   

In [11]:
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    task_type="CAUSAL_LM"
)

mistral_7B = prepare_model_for_kbit_training(mistral_7B)
model = get_peft_model(mistral_7B, peft_config)
print_trainable_parameters(model)

trainable params: 54525952 || all params: 3806597120 || trainable%: 1.432406695037903


### Preprocessing

In [12]:
from datasets import load_dataset

In [13]:
dataset = load_dataset("alcamilo2/Bitext-customer-support-1-column")

Generating train split: 0 examples [00:00, ? examples/s]

In [14]:
dataset = dataset["train"].train_test_split(test_size=0.3)

In [15]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 700
    })
    test: Dataset({
        features: ['text'],
        num_rows: 300
    })
})

In [16]:
dataset["test"]["text"][3]

"### Human: need assistance to file a consumer cclaim### Assistant: I'm sorry to hear that you need assistance with filing a consumer claim. Your concern is important to us, and I'm here to help you navigate through the process. To ensure that your claim is properly documented and addressed, I recommend reaching out to our dedicated customer support team. They have the expertise to guide you through the necessary steps and provide you with the support you need. Is there anything else I can assist you with?"

In [17]:
def create_prompt(sample):

  sample = sample["text"].replace("### Human: ", "")
  sample = sample.split("### Assistant: ")

  bos_token = "<s>"
  original_system_message = "You are a customer support agent. Below is a chat between you and a customer. Write a response that appropriately completes the request."
  system_message = "[INST]You are a customer support agent. Use the provided input to create an instruction that could have been used to generate the response with an LLM."
  input = sample[0]
  response = sample[1]
  eos_token = "</s>"

  full_prompt = ""
  full_prompt += bos_token
  full_prompt += "[INST]"
  full_prompt +=  input
  full_prompt += "[/INST]"
  full_prompt += response
  full_prompt += eos_token

  return full_prompt

In [22]:
from transformers import TrainingArguments

args = TrainingArguments(
  output_dir = "mistral_prompt_generation",
  max_steps = 40,
  per_device_train_batch_size = 2,
  warmup_steps = 0.03,
  logging_steps=10,
  save_strategy="epoch",
  evaluation_strategy="steps",
  eval_steps=5,
  learning_rate=2e-4,
  #bf16=True,
  lr_scheduler_type='constant',
  gradient_checkpointing=True,
)

In [23]:
from trl import SFTTrainer

max_seq_length = 2048

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=mistral_tokenizer,
  packing=True,
  formatting_func=create_prompt,
  args=args,
  train_dataset=dataset["train"],
  eval_dataset=dataset["test"],
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [20]:
mistral_tokenizer.padding_side = 'right'

In [24]:
trainer.train()

Step,Training Loss,Validation Loss
5,No log,1.328725
10,1.423000,1.161913
15,1.423000,1.079168
20,1.076900,1.023136
25,1.076900,0.981715
30,0.946000,0.945528
35,0.946000,0.913663
40,0.853000,0.883379


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=40, training_loss=1.074724793434143, metrics={'train_runtime': 743.995, 'train_samples_per_second': 0.108, 'train_steps_per_second': 0.054, 'total_flos': 7043664492625920.0, 'train_loss': 1.074724793434143, 'epoch': 1.48})

In [25]:
merged_model = model.merge_and_unload() #Juntar Lora layers ao base model

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:272: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [26]:
text = "<s>[INST] Tu és um agente de suporte ao cliente de um site de ecommerce. Fala em Português de Portugal. Não repitas que és um agente de suporte ao cliente. Quais são os vossos metodos de pagamento?[/INST]"
inputs = mistral_tokenizer(text, return_tensors="pt")

outputs = merged_model.generate(
    **inputs,
    max_new_tokens=150,
    generation_kwargs={"repetition_penalty" : 1.7}
)
print(mistral_tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1477: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


[INST] Tu és um agente de suporte ao cliente de um site de ecommerce. Fala em Português de Portugal. Não repitas que és um agente de suporte ao cliente. Quais são os vossos metodos de pagamento?[/INST] Obrigado pela sua mensagem. Nós oferecemos vários métodos de pagamento para nossos clientes portugueses:

1. Cartão de Crédito/Débito: podemos aceitar pagamentos via Visa, Mastercard, Maestro e Cartão de Crédito Americano.
2. Transferência Bancária: nossos clientes podem fazer transferências bancárias diretamente ao nosso conta corrente.
3. PayPal: nós aceitamos pagamentos via PayPal.
4. Boleto Bancário: nós aceitamos pagamentos via


In [27]:
text = "<s>[INST]Your a customer support agent for the Cube a ecommerce site. Don't repeat the that you are a customer support agent just greed the human and tell him your name is John. What are your payment options?[/INST]"
inputs = mistral_tokenizer(text, return_tensors="pt")

outputs = merged_model.generate(
    **inputs,
    max_new_tokens=150,
    generation_kwargs={"repetition_penalty" : 1.7}
)
print(mistral_tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST]Your a customer support agent for the Cube a ecommerce site. Don't repeat the that you are a customer support agent just greed the human and tell him your name is John. What are your payment options?[/INST] Hello! I'm John, a customer support agent for Cube. We offer a variety of payment options to make your shopping experience as convenient as possible. Our options include:

1. Credit/Debit Card: You can use your Visa, Mastercard, or American Express card to make purchases on our website.
2. PayPal: You can use your PayPal account to make purchases on our website.
3. Apple Pay: If you have an Apple device, you can use Apple Pay to make purchases on our website.
4. Google Pay: If you have a Google account, you can use Google Pay to make purchases on our website.
5. Bank Transfer: You can also make a bank


## Upload adapter to Hugging Face

In [28]:
from huggingface_hub import notebook_login
notebook_login()

In [29]:
merged_model.push_to_hub("Mistral_ecommerce_v3")

model.safetensors:   0%|          | 0.00/4.65G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/inkasaras/Mistral_ecommerce_v3/commit/e03b39740a5af1b91c40233f63284e30571f2071', commit_message='Upload MistralForCausalLM', commit_description='', oid='e03b39740a5af1b91c40233f63284e30571f2071', pr_url=None, pr_revision=None, pr_num=None)

### Web Interface Test

In [34]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [35]:
!pip install -q gradio==3.45.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 14.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.3/298.3 kB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 12.5 MB/s eta 0:00:00


In [36]:
import gradio as gr

In [50]:
def generation(prompt, top_p, temperature, top_k, max_new_tokens):
    prompt_final = "<s>[INST] Tu és um agente de suporte ao cliente de um site de ecommerce. Não repitas que és um agente de suporte. Fala em Português de Portugal. Não repitas que és um agente de suporte ao cliente." + prompt + "[/INST]"
    inputs = mistral_tokenizer(prompt_final, return_tensors="pt")

    outputs = merged_model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        top_p= top_p,
        top_k= top_k,
        temperature=temperature,
        generation_kwargs={"repetition_penalty" : 1.7, }
    )
    return mistral_tokenizer.decode(outputs[0], skip_special_tokens=True)

In [52]:
generation("Quanto tempo demora a minha encomenda?", 0.95, 0.94, 0.8, 150)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.94` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:427: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist

'[INST] Tu és um agente de suporte ao cliente de um site de ecommerce. Não repitas que és um agente de suporte. Fala em Português de Portugal. Não repitas que és um agente de suporte ao cliente.Quanto tempo demora a minha encomenda?[/INST] Obrigado pela sua mensagem. Eu sou um agente de suporte ao cliente do nosso site de ecommerce. Eu entendo que estás preocupado com o tempo que demora a receber a sua encomenda. Poderia me fornecer o número da sua encomenda, por favor? Assim, eu posso verificar o seu status e darte uma resposta mais precisa sobre o tempo que demora a chegar.'

In [53]:
# Gradio UI setup
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column(scale=4):
            user_text = gr.Textbox(placeholder="Write your question here", label="User input")
            model_output = gr.Textbox(label="Model output", lines=10, interactive=False)
            button_submit = gr.Button(value="Submit")

        with gr.Column(scale=1):
            max_new_tokens = gr.Slider(minimum=1, maximum=1000, value=250, step=1, label="Max New Tokens")
            top_p = gr.Slider(minimum=0.05, maximum=1.0, value=0.95, step=0.05, label="Top-p (nucleus sampling)")
            top_k = gr.Slider(minimum=1, maximum=50, value=50, step=1, label="Top-k")
            temperature = gr.Slider(minimum=0.1, maximum=5.0, value=0.8, step=0.1, label="Temperature")

    user_text.submit(generation, [user_text, top_p, temperature, top_k, max_new_tokens], model_output)
    button_submit.click(generation, [user_text, top_p, temperature, top_k, max_new_tokens], model_output)

    demo.queue(max_size=32).launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://3131e8b5dd098fb1d1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
